In [1]:
from pathlib import Path
import pandas as pd

# Path 정리
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_yolo_finetuning/")
CSV_PATH = DATA_DIR / "metadata.csv"

# CSV 불러오기
df = pd.read_csv(CSV_PATH)

target = 1                              # 원하는 행 인덱스 설정

COMMON_PATH = df.loc[target,"common_path"]   # COMMON_PATH 추출
VIDEO_PTH = df.loc[target,"video_path"]
FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH          # 프레임 디렉토리 경로 설정
KPT_DIR = DATA_DIR / "2_KEYPOINTS" / COMMON_PATH        # 키포인트 디렉토리 경로 설정
MP4_DIR = DATA_DIR / "3_MP4" / f"{COMMON_PATH}.mp4"              # MP4 디렉토리 경로 설정    
INTERP_DIR = DATA_DIR / "4_INTERP_DATA" /COMMON_PATH    # 보간 데이터 디렉토리 경로 설정
YOLO_DIR = DATA_DIR / "5_YOLO_TXT" / COMMON_PATH     # YOLO TXT 디렉토리 경로 설정

OUTPUT_PATH = DATA_DIR / "test"

# 경로 출력
print(" BASE_DIR:", BASE_DIR,"\n FRAME_DIR:", FRAME_DIR, "\n KPT_DIR:", KPT_DIR, "\n MP4_DIR:", MP4_DIR, "\n INTERP_DIR:", INTERP_DIR)
print("IS_TRAIN:", df.loc[target,"is_train"])

In [2]:
import sys
sys.path.append(str(BASE_DIR))
from funcs.data_utils import convert_json_to_yolo_kpt_fixed, create_yolo_dataset_structure

# JSON2TXT
파이썬 파일 runner로 변경 완료

In [ ]:
# import json
# import cv2
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm

# # ==========================================
# # 1. 경로 설정 (기존과 동일)
# # ==========================================
# DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
# CSV_PATH = DATA_DIR / "metadata.csv"

# df = pd.read_csv(CSV_PATH)
# target_row = df[df['is_train'] == True].iloc[1]

# COMMON_PATH = target_row['common_path']
# FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH
# INTERP_DIR = DATA_DIR / "4_INTERP_DATA" / COMMON_PATH
# YOLO_DIR = DATA_DIR / "5_YOLO_TXT" / COMMON_PATH

# print(f"🧪 [TEST START] 타겟 경로: {COMMON_PATH}")
# print(f"📂 저장 위치: {YOLO_DIR}")

# # ==========================================
# # 3. 실행
# # ==========================================

# YOLO_DIR.mkdir(parents=True, exist_ok=True)

# img_files = list(FRAME_DIR.glob("*.jpg")) + list(FRAME_DIR.glob("*.png"))

# if not img_files:
#     print("❌ 이미지가 없습니다.")
# else:
#     sample_img = cv2.imread(str(img_files[0]))
#     H, W = sample_img.shape[:2]
#     print(f"📏 Image Size: W={W}, H={H}")

#     json_files = list(INTERP_DIR.glob("*.json"))
#     success_count = 0
    
#     print(f"🚀 변환 시작 ({len(json_files)}개 파일)...")

#     for json_file in tqdm(json_files):
#         txt_file = YOLO_DIR / f"{json_file.stem}.txt"
#         if convert_json_to_yolo_kpt_fixed(json_file, txt_file, W, H):
#             success_count += 1

#     print(f"\n✅ 완료: {success_count}개 생성됨.")
    
#     if success_count > 0:
#         sample_txt = list(YOLO_DIR.glob("*.txt"))[0]
#         with open(sample_txt, 'r') as f:
#             print(f"\n📄 [샘플 데이터] {sample_txt.name}\n{f.read()}")
#     else:
#         print("⚠️ 생성된 TXT 파일이 없습니다.")

## 제대로 생성됐는지 확인하는 코드
확인 완료

In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# from pathlib import Path

# # ==========================================
# # ⚙️ 설정 및 스켈레톤 정의
# # ==========================================
# # 우리가 추출한 5~16번 키포인트 (총 12개)에 대한 내부 인덱스 매핑
# # (Original ID -> Local List Index)
# # 5:L-Sh, 6:R-Sh, 7:L-Elb, 8:R-Elb, 9:L-Wri, 10:R-Wri
# # 11:L-Hip, 12:R-Hip, 13:L-Knee, 14:R-Knee, 15:L-Ank, 16:R-Ank

# # 시각화를 위한 연결 정보 (Local Index 기준 0~11)
# SKELETON_CONNECTIONS = [
#     (0, 1),   # Shoulders
#     (0, 2), (2, 4), # Left Arm
#     (1, 3), (3, 5), # Right Arm
#     (0, 6), (1, 7), # Torso (Shoulder -> Hip)
#     (6, 7),   # Hips
#     (6, 8), (8, 10), # Left Leg
#     (7, 9), (9, 11)  # Right Leg
# ]

# def visualize_yolo_overlay(img_path, txt_path):
#     """
#     이미지와 YOLO txt 파일을 읽어 BBox와 Keypoints를 그립니다.
#     """
#     # 1. 이미지 로드
#     img = cv2.imread(str(img_path))
#     if img is None:
#         print(f"❌ 이미지를 찾을 수 없습니다: {img_path}")
#         return
    
#     # BGR -> RGB 변환 (Matplotlib용)
#     img_vis = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     H, W = img_vis.shape[:2]

#     # 2. TXT 파일 읽기
#     if not txt_path.exists():
#         print(f"❌ 라벨 파일을 찾을 수 없습니다: {txt_path}")
#         return

#     with open(txt_path, 'r') as f:
#         lines = f.readlines()

#     print(f"🔍 감지된 객체 수: {len(lines)}")

#     # 3. 파싱 및 그리기
#     for line in lines:
#         data = list(map(float, line.strip().split()))
        
#         # 포맷: <class> <cx> <cy> <w> <h> <kpt1_x> <kpt1_y> <kpt1_v> ...
#         cls_id = int(data[0])
#         cx, cy, w, h = data[1:5]
#         kpts = data[5:]

#         # --- A. Bounding Box 그리기 ---
#         # Normalized Center XYWH -> Pixel Top-Left XY
#         x1 = int((cx - w / 2) * W)
#         y1 = int((cy - h / 2) * H)
#         x2 = int((cx + w / 2) * W)
#         y2 = int((cy + h / 2) * H)

#         # 녹색 박스 (두께 2)
#         cv2.rectangle(img_vis, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         cv2.putText(img_vis, f"Person", (x1, y1 - 10), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#         # --- B. Keypoints 그리기 ---
#         # kpts 리스트를 (x, y, v) 묶음으로 변환
#         points = []
#         for i in range(0, len(kpts), 3):
#             kx, ky, kv = kpts[i], kpts[i+1], kpts[i+2]
            
#             # 픽셀 좌표 변환
#             px, py = int(kx * W), int(ky * H)
#             points.append((px, py))

#             # 점 그리기 (빨간색, 가시성 있는 경우만)
#             if kx > 0 and ky > 0: # 혹은 kv > 0
#                 cv2.circle(img_vis, (px, py), 4, (255, 0, 0), -1) # Red Dot

#         # --- C. Skeleton 연결선 그리기 ---
#         for i, j in SKELETON_CONNECTIONS:
#             # 인덱스 범위 체크 및 좌표 유효성 체크
#             if i < len(points) and j < len(points):
#                 pt1 = points[i]
#                 pt2 = points[j]
                
#                 # 둘 다 (0,0)이 아닐 때만 선 긋기
#                 if (pt1[0] > 0 and pt1[1] > 0) and (pt2[0] > 0 and pt2[1] > 0):
#                     cv2.line(img_vis, pt1, pt2, (0, 255, 255), 2) # Yellow Line

#     # 4. 출력 (Matplotlib)
#     plt.figure(figsize=(12, 12))
#     plt.imshow(img_vis)
#     plt.axis('off')
#     plt.title(f"Overlay Result: {txt_path.name}")
#     plt.show()

# # ==========================================
# # 🚀 실행
# # ==========================================
# # 앞서 생성된 파일이 있다고 가정합니다.
# # YOLO_DIR 변수가 이전 코드에서 정의되어 있어야 합니다.

# # 예시: 폴더 내 첫 번째 txt 파일과 매칭되는 이미지 찾기
# txt_files = list(YOLO_DIR.glob("*.txt"))

# if txt_files:
#     test_txt = txt_files[200]
#     # 이미지 이름은 txt 파일명과 같다고 가정 (확장자만 다름)
#     # 이미지 확장자를 모르니 jpg, png 시도
#     test_img = FRAME_DIR / f"{test_txt.stem}.jpg"
#     if not test_img.exists():
#         test_img = FRAME_DIR / f"{test_txt.stem}.png"

#     print(f"🖼️ 이미지: {test_img}")
#     print(f"📝 라벨: {test_txt}")
    
#     visualize_yolo_overlay(test_img, test_txt)
# else:
#     print("❌ 확인할 TXT 파일이 없습니다.")

# datasset 구성 
symlink 사용해서 용량을 줄임. dataset 구조를 맞추기 위해 사용하는 코드  
funcs에 `create_yolo_dataset_structure(df, dataset_dir, data_dir, step=30)` 함수로 생성했음


In [ ]:
# import pandas as pd
# import yaml
# import os
# import shutil
# from pathlib import Path

# # ==========================================
# # 1. 설정 및 경로
# # ==========================================
# # 원본 데이터 경로 (요청하신 경로 고정)
# DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
# CSV_PATH = DATA_DIR / "metadata.csv"

# # 테스트용 결과가 저장될 경로
# TEST_DATASET_DIR = DATA_DIR / "6_YOLO_TRAINING_DATA/v1.0"

# # ==========================================
# # 3. 메인 실행부 (2개 샘플 추출 및 실행)
# # ==========================================
# if __name__ == "__main__":
#     # 1. 메타데이터 로드
#     if not CSV_PATH.exists():
#         print(f"❌ CSV 파일을 찾을 수 없습니다: {CSV_PATH}")
#     else:
#         full_df = pd.read_csv(CSV_PATH)

#         # 2. 테스트용 샘플 데이터 추출 (Train 1개 + Val 1개)
#         try:
#             sample_rows = []
            
#             # Train 샘플 (첫 번째 행)
#             train_rows = full_df[full_df['is_train'] == True]
#             if not train_rows.empty:
#                 sample_rows.append(train_rows.iloc[0])
            
#             # Val 샘플 (첫 번째 행)
#             if 'is_val' in full_df.columns:
#                 val_rows = full_df[full_df['is_val'] == True]
#                 if not val_rows.empty:
#                     sample_rows.append(val_rows.iloc[0])
#             else:
#                 print("⚠️ 'is_val' 컬럼이 없어 Train 데이터만 테스트합니다.")

#             if sample_rows:
#                 test_df = pd.DataFrame(sample_rows)
#                 print(f"🧪 테스트 대상: 총 {len(test_df)}개 폴더")
                
#                 # 3. 함수 실행
#                 create_yolo_dataset_structure(test_df, TEST_DATASET_DIR, DATA_DIR, step=30)
                
#                 # 4. 결과 확인
#                 print("\n📂 생성된 폴더 구조 확인:")
#                 for root, dirs, files in os.walk(TEST_DATASET_DIR):
#                     level = root.replace(str(TEST_DATASET_DIR), '').count(os.sep)
#                     indent = ' ' * 4 * (level)
#                     print(f"{indent}{os.path.basename(root)}/")
#                     for f in files[:3]: 
#                         print(f"{indent}    {f}")
#                     if len(files) > 3:
#                         print(f"{indent}    ... ({len(files)-3} more files)")
#             else:
#                 print("❌ 테스트할 조건에 맞는 데이터가 없습니다.")
                
#         except Exception as e:
#             print(f"❌ 실행 중 오류 발생: {e}")

# YOLO finetuning

In [ ]:
# import os
# import yaml
# from pathlib import Path
# from ultralytics import YOLO
# import wandb
# from dotenv import load_dotenv

# # ---------------------------------------------------------
# # 1. 환경 설정 및 데이터 준비
# # ---------------------------------------------------------
# ENV_PATH = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_yolo_finetuning/.env"
# load_dotenv(ENV_PATH)
# WANDB_API_KEY = os.getenv("WANDB_API_KEY")

# CONFIG_PATH = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_yolo_finetuning/exp_v1.yaml"
# with open(CONFIG_PATH, 'r') as f:
#     cfg = yaml.safe_load(f)

# # ---------------------------------------------------------
# # 🛠️ 함수: 경로 유동성 해결 & 메타데이터 추출
# # ---------------------------------------------------------
# def update_data_yaml_and_get_info(yaml_path):
#     path_obj = Path(yaml_path)
#     with open(path_obj, 'r') as f:
#         data_cfg = yaml.safe_load(f)
    
#     current_data_dir = path_obj.parent.resolve()
#     print(f"🔄 데이터 경로 갱신: {data_cfg.get('path')} -> {current_data_dir}")
#     data_cfg['path'] = str(current_data_dir)
    
#     step_info = data_cfg.get('sampling_step', 'Unknown')
#     print(f"ℹ️ 데이터셋 Sampling Step: {step_info}")

#     with open(path_obj, 'w') as f:
#         yaml.dump(data_cfg, f, sort_keys=False)
#     return str(path_obj), step_info

# target_data_yaml = cfg['data']['config_path']
# fixed_data_yaml, dataset_step = update_data_yaml_and_get_info(target_data_yaml)

# # ---------------------------------------------------------
# # 🚀 WandB 초기화
# # ---------------------------------------------------------
# PROJECT = cfg['project_name']
# RUN_NAME = cfg['run_name']

# if cfg['logging']['use_wandb'] and WANDB_API_KEY:
#     try:
#         wandb.login(key=WANDB_API_KEY)
#         wandb_config = cfg.copy()
#         wandb_config['dataset'] = {'sampling_step': dataset_step, 'yaml_path': fixed_data_yaml}

#         wandb.init(
#             project=PROJECT,
#             name=RUN_NAME,
#             config=wandb_config,
#             resume="allow",
#             dir=cfg['output']['base_dir']
#         )
#         print(f"✅ WandB 초기화 성공 (Pose Task | Step: {dataset_step})")
#     except Exception as e:
#         print(f"⚠️ WandB 초기화 실패: {e}")

# # ---------------------------------------------------------
# # 🛠️ [핵심] 커스텀 WandB 콜백 (Pose Metrics 포함)
# # ---------------------------------------------------------
# def on_train_epoch_end(trainer):
#     """
#     매 에폭 종료 시 실행. Pose Loss, Box Loss, mAP 등을 모두 기록합니다.
#     """
#     if wandb.run:
#         # trainer.metrics 안에는 'pose_loss', 'box_loss' 등이 자동으로 포함됩니다.
#         wandb.log(trainer.metrics)
#         wandb.log({"epoch": trainer.epoch + 1})

# # ---------------------------------------------------------
# # 🤖 스마트 모델 로드 & 이어하기 (Pose Model 전용)
# # ---------------------------------------------------------
# CHECKPOINT_DIR = os.path.join(cfg['output']['base_dir'], RUN_NAME, 'weights')
# LAST_PT_PATH = os.path.join(CHECKPOINT_DIR, 'last.pt')
# BASE_MODEL_PATH = cfg['model']['base_path']

# resume_status = False

# # 1. 이어하기 (last.pt) 체크
# if os.path.exists(LAST_PT_PATH):
#     print(f"🔄 [Resume] 이전 학습 체크포인트를 발견했습니다: {LAST_PT_PATH}")
#     model = YOLO(LAST_PT_PATH)
#     resume_status = True

# # 2. 처음 시작 (설정 파일의 모델 경로 사용)
# elif os.path.exists(BASE_MODEL_PATH):
#     print(f"🆕 [Start] 설정된 Pose 모델을 로드합니다: {BASE_MODEL_PATH}")
#     model = YOLO(BASE_MODEL_PATH)
#     resume_status = False

# # 3. 파일 없음 (자동 다운로드 - Pose 버전 명시)
# else:
#     print(f"⚠️ [Download] 모델 파일을 찾을 수 없어 'yolo11n-pose.pt'를 다운로드합니다.")
#     # 사용자가 실수로 일반 모델을 적었더라도, 파일이 없으면 확실하게 pose 모델을 받도록 처리
#     model = YOLO("yolo11n-pose.pt") 
#     resume_status = False

# # ---------------------------------------------------------
# # 🔗 콜백 등록 및 학습 시작
# # ---------------------------------------------------------
# # 커스텀 콜백 등록
# model.add_callback("on_train_epoch_end", on_train_epoch_end)
# print("✅ 커스텀 WandB 콜백 등록 완료")

# print(f"\n🔥 Pose Estimation 학습 시작: {RUN_NAME} (Resume: {resume_status})")

# model.train(
#     data=fixed_data_yaml,
#     project=cfg['output']['base_dir'], 
#     name=RUN_NAME,
#     resume=resume_status,
#     plots=True,
#     **cfg['train'] 
# )

# if wandb.run:
#     wandb.finish()